In [40]:
import pandas as pd
import numpy as np

df = pd.read_csv("SLU Opportunity Wise Data.csv")
print(df.head())


  Learner SignUp DateTime                  Opportunity Id  \
0     06/14/2023 12:30:35  00000000-0GN2-A0AY-7XK8-C5FZPP   
1     05/01/2023 05:29:16  00000000-0GN2-A0AY-7XK8-C5FZPP   
2     04/09/2023 20:35:08  00000000-0GN2-A0AY-7XK8-C5FZPP   
3     08/29/2023 05:20:03  00000000-0GN2-A0AY-7XK8-C5FZPP   
4     01/06/2023 15:26:36  00000000-0GN2-A0AY-7XK8-C5FZPP   

                                    Opportunity Name Opportunity Category  \
0  Career Essentials: Getting Started with Your P...               Course   
1  Career Essentials: Getting Started with Your P...               Course   
2  Career Essentials: Getting Started with Your P...               Course   
3  Career Essentials: Getting Started with Your P...               Course   
4  Career Essentials: Getting Started with Your P...               Course   

  Opportunity End Date        First Name Date of Birth  Gender        Country  \
0  06/29/2024 18:52:39             Faria    01/12/2001  Female       Pakistan   
1  06/29

In [41]:
df.head()
df.shape
df.columns
print(df.isnull().sum())
df.describe()

Learner SignUp DateTime       0
Opportunity Id                0
Opportunity Name              0
Opportunity Category          0
Opportunity End Date          0
First Name                    0
Date of Birth                 0
Gender                        0
Country                       0
Institution Name              5
Current/Intended Major        5
Entry created at              0
Status Description            0
Status Code                   0
Apply Date                    0
Opportunity Start Date     3794
dtype: int64


,Status Code
count,8558.000000
mean,1052.225987
std,21.665207
min,1010.000000
25%,1030.000000
50%,1050.000000
75%,1070.000000
max,1120.000000


In [42]:
df.dtypes

Learner SignUp DateTime    object
Opportunity Id             object
Opportunity Name           object
Opportunity Category       object
Opportunity End Date       object
First Name                 object
Date of Birth              object
Gender                     object
Country                    object
Institution Name           object
Current/Intended Major     object
Entry created at           object
Status Description         object
Status Code                 int64
Apply Date                 object
Opportunity Start Date     object
dtype: object

Standardize Date Formats

In [43]:
date_cols = [
    'Learner SignUp DateTime', 
    'Opportunity End Date', 
    'Date of Birth', 
    'Entry created at', 
    'Apply Date', 
    'Opportunity Start Date'
]
df[date_cols] = df[date_cols].apply(pd.to_datetime, errors='coerce')

In [44]:
invalid_dates = df[df['Learner SignUp DateTime'].isna()]
print(invalid_dates[['Learner SignUp DateTime']])

     Learner SignUp DateTime
51                       NaT
93                       NaT
139                      NaT
155                      NaT
178                      NaT
...                      ...
8477                     NaT
8494                     NaT
8509                     NaT
8539                     NaT
8548                     NaT

[295 rows x 1 columns]


Removing rows where Learner Signup Date is missing

In [45]:
df=df.dropna(subset=['Learner SignUp DateTime'])

In [46]:
df['Learner SignUp DateTime'].head()

0   2023-06-14 12:30:35
1   2023-05-01 05:29:16
2   2023-04-09 20:35:08
3   2023-08-29 05:20:03
4   2023-01-06 15:26:36
Name: Learner SignUp DateTime, dtype: datetime64[ns]

Opportunity Start Date: Fill missing values with the mode (most common start date) since all entries share the same opportunity ID:

In [47]:
mode_start_date = df['Opportunity Start Date'].mode()[0]
df['Opportunity Start Date'] = df['Opportunity Start Date'].fillna(mode_start_date)
print("handling missing values in Opportunity Start Date")
print(df['Opportunity Start Date'].isnull().sum())

handling missing values in Opportunity Start Date
0


In [48]:
df.dropna(subset=['Apply Date'], inplace=True)

In [49]:
df.shape

(8024, 16)

Standardize Categorical Data
Gender: Standardize values (e.g., Male, Female, Other):

In [50]:
df['Gender'] = df['Gender'].str.strip().replace({'M': 'Male', 'F': 'Female', 'Unknown': 'Other'})


Standardizing Formats (Cleaning Text Fields)

In [51]:
text_columns = ['Current/Intended Major', 'Institution Name', 'Gender']
for col in text_columns:
    df[col] = df[col].astype(str).str.strip().str.lower()

In [52]:
df.duplicated().sum()


np.int64(0)

In [53]:
df.duplicated(subset=['First Name', 'Opportunity Id', 'Apply Date']).sum()

np.int64(0)

In [54]:
print(df.isnull().sum())

Learner SignUp DateTime       0
Opportunity Id                0
Opportunity Name              0
Opportunity Category          0
Opportunity End Date       1175
First Name                    0
Date of Birth                 0
Gender                        0
Country                       0
Institution Name              0
Current/Intended Major        0
Entry created at              0
Status Description            0
Status Code                   0
Apply Date                    0
Opportunity Start Date        0
dtype: int64


In [55]:
df['Institution Name'].head(100)

0                                                  nwihs
1                                            saint louis
3                                 saint louis university
4                                 saint louis university
5                                 saint louis university
                             ...                        
98                                saint louis university
99                                  university of ilorin
100                               ignatius global school
101    mehran uniiversity of engineering and technolo...
102                       delta state university  abraka
Name: Institution Name, Length: 100, dtype: object

In [56]:
df.columns

Index(['Learner SignUp DateTime', 'Opportunity Id', 'Opportunity Name',
       'Opportunity Category', 'Opportunity End Date', 'First Name',
       'Date of Birth', 'Gender', 'Country', 'Institution Name',
       'Current/Intended Major', 'Entry created at', 'Status Description',
       'Status Code', 'Apply Date', 'Opportunity Start Date'],
      dtype='object')

In [57]:
df['Current/Intended Major'].value_counts()

Current/Intended Major
information systems                    1987
computer science                        986
health data science                     339
cyber security                          309
computer information systems            234
                                       ... 
www                                       1
bsc cs                                    1
high school student                       1
computer networking and it security       1
pos service                               1
Name: count, Length: 386, dtype: int64

Making following columns lower case

In [58]:
text_columns = ['Current/Intended Major', 'Institution Name', 'Gender']
for col in text_columns:
    df[col] = df[col].astype(str).str.strip().str.lower()


Handling Inconsistent Categorical Data

In [59]:
gender_mapping = {
    'm': 'male',
    'male': 'male',
    'f': 'female',
    'female': 'female',
    'other': 'other',
    'prefer not to say': 'unknown'
}


In [60]:
df['Gender'].head(100)

0      female
1      female
3      female
4        male
5        male
        ...  
98       male
99     female
100    female
101    female
102    female
Name: Gender, Length: 100, dtype: object

First letter of each word to uppercase and the rest to lowercase

In [61]:
df['Gender'] = df['Gender'].str.title()
df['Current/Intended Major'] = df['Current/Intended Major'].str.title()
df['Institution Name'] = df['Institution Name'].str.title()
df['First Name'] = df['First Name'].str.title()


Selects columns from the DataFrame. Then, it removes duplicate rows based on the combination of values in these three columns

In [62]:
print(df[['Gender', 'Current/Intended Major', 'Institution Name']].drop_duplicates().head(10))

    Gender               Current/Intended Major  \
0   Female                            Radiology   
1   Female                  Information Systems   
3   Female                  Information Systems   
4     Male                     Computer Science   
5     Male               Mechanical Engineering   
6     Male     Computer Science And Engineering   
7   Female              Artificial Intelligence   
8     Male  Robotics And Automation Engineering   
9   Female                     Computer Science   
10  Female                   Data Visualization   

                       Institution Name  
0                                 Nwihs  
1                           Saint Louis  
3                Saint Louis University  
4                Saint Louis University  
5                Saint Louis University  
6      Illinois Institute Of Technology  
7   Gems New Millennium School Al Khail  
8                        Ashoka Academy  
9      Illinois Institute Of Technology  
10           Feder

Check for unusual names: very short, contain digits or special characters

In [63]:
unusual_names = df['First Name'].dropna().unique()
suspicious = [name for name in unusual_names if len(name) <= 2 or any(char.isdigit() or not char.isalnum() for char in name)]

# Print first few suspicious names
print(suspicious[:20])


['Amrutha Varshini', 'Vinay Varshith', 'Marian Morenike', 'Sri Vasista', 'Nandam Venkata Naga Vijay', 'Sai Ram Pulla Reddy', 'Usman Bashir', 'Rp19-Ee-418', 'Lakshmi Naga Sushmit', '19-R-0413', 'Emmanuel Adu', 'Vellala Linga Bhanu Prakash', 'Marzia Maheen', 'Lokesh Reddy', 'Jan Muhammad', 'Heldana Shiferaw', 'Sk', 'Shreeja Yadav', 'Chandan Singh', 'Sri Lalitha']


In [64]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 8024 entries, 0 to 8557
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Learner SignUp DateTime  8024 non-null   datetime64[ns]
 1   Opportunity Id           8024 non-null   object        
 2   Opportunity Name         8024 non-null   object        
 3   Opportunity Category     8024 non-null   object        
 4   Opportunity End Date     6849 non-null   datetime64[ns]
 5   First Name               8024 non-null   object        
 6   Date of Birth            8024 non-null   datetime64[ns]
 7   Gender                   8024 non-null   object        
 8   Country                  8024 non-null   object        
 9   Institution Name         8024 non-null   object        
 10  Current/Intended Major   8024 non-null   object        
 11  Entry created at         8024 non-null   datetime64[ns]
 12  Status Description       8024 non-null 

In [65]:
print(df.isnull().sum())

Learner SignUp DateTime       0
Opportunity Id                0
Opportunity Name              0
Opportunity Category          0
Opportunity End Date       1175
First Name                    0
Date of Birth                 0
Gender                        0
Country                       0
Institution Name              0
Current/Intended Major        0
Entry created at              0
Status Description            0
Status Code                   0
Apply Date                    0
Opportunity Start Date        0
dtype: int64


In [66]:
df['Gender'] = df['Gender'].fillna("Unknown")

In [67]:
df.isnull().sum()

Learner SignUp DateTime       0
Opportunity Id                0
Opportunity Name              0
Opportunity Category          0
Opportunity End Date       1175
First Name                    0
Date of Birth                 0
Gender                        0
Country                       0
Institution Name              0
Current/Intended Major        0
Entry created at              0
Status Description            0
Status Code                   0
Apply Date                    0
Opportunity Start Date        0
dtype: int64

In [68]:
df['Opportunity Start Date'] = pd.to_datetime(df['Opportunity Start Date'], errors='coerce')

# Replace each group's 'Opportunity Start Date' with the mode for that group
df['Opportunity Start Date'] = (
    df.groupby('Opportunity Name')['Opportunity Start Date']
    .transform(lambda x: x.mode().iloc[0] if not x.mode().empty else pd.NaT)
)

In [69]:
from datetime import datetime

# Calculate age
df['Age'] = df['Date of Birth'].apply(lambda dob: datetime.now().year - dob.year)

# Display the head to verify the new column
display(df.head())

,Learner SignUp DateTime,Opportunity Id,Opportunity Name,Opportunity Category,Opportunity End Date,First Name,Date of Birth,Gender,Country,Institution Name,Current/Intended Major,Entry created at,Status Description,Status Code,Apply Date,Opportunity Start Date,Age
0,2023-06-14 12:30:35,00000000-0GN2-A0AY-7XK8-C5FZPP,Career Essentials: Getting Started with Your P...,Course,2024-06-29 18:52:39,Faria,2001-01-12,Female,Pakistan,Nwihs,Radiology,2024-03-11 12:01:41,Started,1080,2023-06-14 12:36:09,2022-11-03 18:30:39,24
1,2023-05-01 05:29:16,00000000-0GN2-A0AY-7XK8-C5FZPP,Career Essentials: Getting Started with Your P...,Course,2024-06-29 18:52:39,Poojitha,2000-08-16,Female,India,Saint Louis,Information Systems,2024-03-11 12:01:41,Started,1080,2023-05-01 06:08:21,2022-11-03 18:30:39,25
3,2023-08-29 05:20:03,00000000-0GN2-A0AY-7XK8-C5FZPP,Career Essentials: Getting Started with Your P...,Course,2024-06-29 18:52:39,Amrutha Varshini,1999-11-01,Female,United States,Saint Louis University,Information Systems,2024-03-11 12:01:41,Team Allocated,1070,2023-10-09 22:02:42,2022-11-03 18:30:39,26
4,2023-01-06 15:26:36,00000000-0GN2-A0AY-7XK8-C5FZPP,Career Essentials: Getting Started with Your P...,Course,2024-06-29 18:52:39,Vinay Varshith,2000-04-19,Male,United States,Saint Louis University,Computer Science,2024-03-11 12:01:41,Started,1080,2023-01-06 15:40:10,2022-11-03 18:30:39,25
5,2024-03-02 19:16:07,00000000-0GN2-A0AY-7XK8-C5FZPP,Career Essentials: Getting Started with Your P...,Course,2024-06-29 18:52:39,Mor,1996-05-12,Male,India,Saint Louis University,Mechanical Engineering,2024-03-11 12:01:41,Waitlisted,1040,2024-03-02 20:30:35,2022-11-03 18:30:39,29


In [70]:
# Identify unusual names: very short, contain digits or special characters
unusual_names = df['First Name'].dropna().unique()
suspicious = [name for name in unusual_names if len(name.replace(" ", "")) <= 2 or any(char.isdigit() or not char.isalnum() and char != ' ' for char in name)]

# Print first few suspicious names
print("Suspicious names found:")
print(suspicious[:20])

Suspicious names found:
['Rp19-Ee-418', '19-R-0413', 'Sk', 'Nữ', 'Manish3Test3', 'Sp', 'Habib.', 'E', 'गरिमा', 'Om', 'V', 'Md', 'Sana_', 'Navnath3Test3', 'K', 'Manmeet3Test3', 'Am014', '20 012', '11', 'Mr']


In [71]:
# Drop rows with suspicious names
df = df[~df['First Name'].isin(suspicious)]

# Display the shape of the DataFrame to see how many rows were dropped
print(f"Shape of DataFrame after dropping rows with suspicious names: {df.shape}")

# Display the head of the DataFrame to see the updated data
display(df.head())

Shape of DataFrame after dropping rows with suspicious names: (7947, 17)


,Learner SignUp DateTime,Opportunity Id,Opportunity Name,Opportunity Category,Opportunity End Date,First Name,Date of Birth,Gender,Country,Institution Name,Current/Intended Major,Entry created at,Status Description,Status Code,Apply Date,Opportunity Start Date,Age
0,2023-06-14 12:30:35,00000000-0GN2-A0AY-7XK8-C5FZPP,Career Essentials: Getting Started with Your P...,Course,2024-06-29 18:52:39,Faria,2001-01-12,Female,Pakistan,Nwihs,Radiology,2024-03-11 12:01:41,Started,1080,2023-06-14 12:36:09,2022-11-03 18:30:39,24
1,2023-05-01 05:29:16,00000000-0GN2-A0AY-7XK8-C5FZPP,Career Essentials: Getting Started with Your P...,Course,2024-06-29 18:52:39,Poojitha,2000-08-16,Female,India,Saint Louis,Information Systems,2024-03-11 12:01:41,Started,1080,2023-05-01 06:08:21,2022-11-03 18:30:39,25
3,2023-08-29 05:20:03,00000000-0GN2-A0AY-7XK8-C5FZPP,Career Essentials: Getting Started with Your P...,Course,2024-06-29 18:52:39,Amrutha Varshini,1999-11-01,Female,United States,Saint Louis University,Information Systems,2024-03-11 12:01:41,Team Allocated,1070,2023-10-09 22:02:42,2022-11-03 18:30:39,26
4,2023-01-06 15:26:36,00000000-0GN2-A0AY-7XK8-C5FZPP,Career Essentials: Getting Started with Your P...,Course,2024-06-29 18:52:39,Vinay Varshith,2000-04-19,Male,United States,Saint Louis University,Computer Science,2024-03-11 12:01:41,Started,1080,2023-01-06 15:40:10,2022-11-03 18:30:39,25
5,2024-03-02 19:16:07,00000000-0GN2-A0AY-7XK8-C5FZPP,Career Essentials: Getting Started with Your P...,Course,2024-06-29 18:52:39,Mor,1996-05-12,Male,India,Saint Louis University,Mechanical Engineering,2024-03-11 12:01:41,Waitlisted,1040,2024-03-02 20:30:35,2022-11-03 18:30:39,29


In [72]:
# Drop rows with missing 'Opportunity End Date'
df.dropna(subset=['Opportunity End Date'], inplace=True)

# Display the shape of the DataFrame after dropping
print(f"Shape of DataFrame after dropping rows with missing 'Opportunity End Date': {df.shape}")

# Display the head to see the updated data
display(df.head())

Shape of DataFrame after dropping rows with missing 'Opportunity End Date': (6778, 17)


,Learner SignUp DateTime,Opportunity Id,Opportunity Name,Opportunity Category,Opportunity End Date,First Name,Date of Birth,Gender,Country,Institution Name,Current/Intended Major,Entry created at,Status Description,Status Code,Apply Date,Opportunity Start Date,Age
0,2023-06-14 12:30:35,00000000-0GN2-A0AY-7XK8-C5FZPP,Career Essentials: Getting Started with Your P...,Course,2024-06-29 18:52:39,Faria,2001-01-12,Female,Pakistan,Nwihs,Radiology,2024-03-11 12:01:41,Started,1080,2023-06-14 12:36:09,2022-11-03 18:30:39,24
1,2023-05-01 05:29:16,00000000-0GN2-A0AY-7XK8-C5FZPP,Career Essentials: Getting Started with Your P...,Course,2024-06-29 18:52:39,Poojitha,2000-08-16,Female,India,Saint Louis,Information Systems,2024-03-11 12:01:41,Started,1080,2023-05-01 06:08:21,2022-11-03 18:30:39,25
3,2023-08-29 05:20:03,00000000-0GN2-A0AY-7XK8-C5FZPP,Career Essentials: Getting Started with Your P...,Course,2024-06-29 18:52:39,Amrutha Varshini,1999-11-01,Female,United States,Saint Louis University,Information Systems,2024-03-11 12:01:41,Team Allocated,1070,2023-10-09 22:02:42,2022-11-03 18:30:39,26
4,2023-01-06 15:26:36,00000000-0GN2-A0AY-7XK8-C5FZPP,Career Essentials: Getting Started with Your P...,Course,2024-06-29 18:52:39,Vinay Varshith,2000-04-19,Male,United States,Saint Louis University,Computer Science,2024-03-11 12:01:41,Started,1080,2023-01-06 15:40:10,2022-11-03 18:30:39,25
5,2024-03-02 19:16:07,00000000-0GN2-A0AY-7XK8-C5FZPP,Career Essentials: Getting Started with Your P...,Course,2024-06-29 18:52:39,Mor,1996-05-12,Male,India,Saint Louis University,Mechanical Engineering,2024-03-11 12:01:41,Waitlisted,1040,2024-03-02 20:30:35,2022-11-03 18:30:39,29


In [73]:
df.shape

(6778, 17)

In [74]:
df.isnull().sum()

Learner SignUp DateTime    0
Opportunity Id             0
Opportunity Name           0
Opportunity Category       0
Opportunity End Date       0
First Name                 0
Date of Birth              0
Gender                     0
Country                    0
Institution Name           0
Current/Intended Major     0
Entry created at           0
Status Description         0
Status Code                0
Apply Date                 0
Opportunity Start Date     0
Age                        0
dtype: int64

In [75]:
df.to_csv('cleaned_data_final.csv', index=False)
print("\n Saved cleaned_data.csv!")


 Saved cleaned_data.csv!


In [76]:
import pandas as pd

# Step 1: Load your data
df = pd.read_csv("cleaned_data_final.csv")

# Step 2: Convert start and end dates to datetime
df['Opportunity Start Date'] = pd.to_datetime(df['Opportunity Start Date'], errors='coerce')
df['Opportunity End Date'] = pd.to_datetime(df['Opportunity End Date'], errors='coerce')

# Step 3: Create internship duration column in days
df['Opportunity Duration (Days)'] = (df['Opportunity End Date'] - df['Opportunity Start Date']).dt.days

# Step 4: Define columns to check for outliers
columns_to_check = ['Age', 'Opportunity Duration (Days)']

# Step 5: Outlier detection and removal
outliers_summary = {}

for col in columns_to_check:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR

    outliers = df[(df[col] < lower) | (df[col] > upper)]
    outliers_summary[col] = {
        'Lower Bound': lower,
        'Upper Bound': upper,
        'Outlier Count': outliers.shape[0]
    }

    # Remove outliers from the DataFrame
    df = df[(df[col] >= lower) & (df[col] <= upper)]

# Step 6: Print summary
for col, stats in outliers_summary.items():
    print(f"\nOutlier Summary for {col}:")
    for k, v in stats.items():
        print(f"{k}: {v}")

# Step 7: Save cleaned dataset
df.to_csv("cleaned_data_final_no_outliers.csv", index=False)
print("\n Cleaned dataset saved as 'cleaned_data_final_no_outliers.csv'")



Outlier Summary for Age:
Lower Bound: 17.0
Upper Bound: 33.0
Outlier Count: 379

Outlier Summary for Opportunity Duration (Days):
Lower Bound: 493.0
Upper Bound: 493.0
Outlier Count: 2736

 Cleaned dataset saved as 'cleaned_data_final_no_outliers.csv'


In [77]:
df.shape

(3663, 18)